# Sentiment Analysis with BERT Sequence Classifier

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Install Dependencies

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
!pip install transformers

!pip install nltk

     |████████████████████████████████| 1.1MB 4.9MB/s 
     |████████████████████████████████| 3.0MB 28.2MB/s 
     |████████████████████████████████| 890kB 49.9MB/s 
     |████████████████████████████████| 1.1MB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4721f055ca5001be95a20320a9db0c209643e6bf6b7f529a1d3d2bb62e324948
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## ML Web App with Streamlit

### Write py file for running with Streamlit

In [ ]:
%%writefile app.py
import pandas as pd
from tqdm.notebook import tqdm
import torch
import os
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from bs4 import BeautifulSoup
nltk.download('stopwords')

#!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader #, RandomSampler, SequentialSampler
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased', 
    do_lower_case=True 
)

import streamlit as st
PAGE_CONFIG = {"page_title":"Sentiment Analysis","page_icon":":smiley:","layout":"centered"}
st.beta_set_page_config(**PAGE_CONFIG)

######################################################################
contractions = {
"aight": "alright",
"ain't": "am not",
"amn't": "am not",
"aren't": "are not",
"can't": "can not",
"cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"daren't": "dare not",
"daresn't": "dare not",
"dasn't": "dare not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"d'ye": "do you",
"e'er": "ever",
"everybody's": "everybody is",
"everyone's": "everyone is",
"finna": "fixing to",
"g'day": "good day",
"gimme": "give me",
"giv'n": "given",
"gonna": "going to",
"gon't": "go not",
"gotta": "got to",
"hadn't": "had not",
"had've": "had have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'dn't've'd": "he would not have had",
"he'll": "he will",
"he's": "he is",
"he've": "he have",
"how'd": "how would",
"howdy": "how do you do",
"how'll": "how will",
"how're": "how are",
"I'll": "I will",
"I'm": "I am",
"I'm'a": "I am about to",
"I'm'o": "I am going to",
"innit": "is it not",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"may've": "may have",
"methinks": "me thinks",
"mightn't": "might not",
"might've": "might have",
"mustn't": "must not",
"mustn't've": "must not have",
"must've": "must have",
"needn't": "need not",
"ne'er": "never",
"o'clock": "of the clock",
"o'er": "over",
"ol'": "old",
"oughtn't": "ought not",
"'s": "is",
"shalln't": "shall not",
"shan't": "shall not",
"she'd": "she would",
"she'll": "she shall",
"she'll": "she will",
"she's": "she has",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"somebody's": "somebody has",
"somebody's": "somebody is",
"someone's": "someone has",
"someone's": "someone is",
"something's": "something has",
"something's": "something is",
"so're": "so are",
"that'll": "that shall",
"that'll": "that will",
"that're": "that are",
"that's": "that has",
"that's": "that is",
"that'd": "that would",
"that'd": "that had",
"there'd": "there had",
"there'd": "there would",
"there'll": "there shall",
"there'll": "there will",
"there're": "there are",
"there's": "there has",
"there's": "there is",
"these're": "these are",
"these've": "these have",
"they'd": "they had",
"they'd": "they would",
"they'll": "they shall",
"they'll": "they will",
"they're": "they are",
"they're": "they were",
"they've": "they have",
"this's": "this has",
"this's": "this is",
"those're": "those are",
"those've": "those have",
"'tis": "it is",
"to've": "to have",
"'twas": "it was",
"wanna": "want to",
"wasn't": "was not",
"we'd": "we had",
"we'd": "we would",
"we'd": "we did",
"we'll": "we shall",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'd": "what did",
"what'll": "what shall",
"what'll": "what will",
"what're": "what are",
"what're": "what were",
"what's": "what has",
"what's": "what is",
"what's": "what does",
"what've": "what have",
"when's": "when has",
"when's": "when is",
"where'd": "where did",
"where'll": "where shall",
"where'll": "where will",
"where're": "where are",
"where's": "where has",
"where's": "where is",
"where's": "where does",
"where've": "where have",
"which'd": "which had",
"which'd": "which would",
"which'll": "which shall",
"which'll": "which will",
"which're": "which are",
"which's": "which has",
"which's": "which is",
"which've": "which have",
"who'd": "who would",
"who'd": "who had",
"who'd": "who did",
"who'd've": "who would have",
"who'll": "who shall",
"who'll": "who will",
"who're": "who are",
"who's": "who has",
"who's": "who is",
"who's": "who does",
"who've": "who have",
"why'd": "why did",
"why're": "why are",
"why's": "why has",
"why's": "why is",
"why's": "why does",
"won't": "will not",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd've": "you all would have",
"y'all'dn't've'd": "you all would not have had",
"y'all're": "you all are",
"you'd": "you had",
"you'd": "you would",
"you'll": "you shall",
"you'll": "you will",
"you're": "you are",
"you're": "you are",
"you've": "you have",
" u ": "you",
" ur ": "your",
" n ": "and"
}

######################################################################

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+" #last part is used to remove special chars and punctuation

def clean_text(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key,value)
        return x
    else:
        return x

def preprocessing(text):
  text = clean_text(text)

  text = cont_to_exp(text)
  
  text = ' '.join(text.split())

  text = BeautifulSoup(text, 'lxml').get_text()

  return text 


def load_model():
  from transformers import BertForSequenceClassification
  model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
  model.to('cuda')
  direc_path = "/content/gdrive/My Drive/Colab Notebooks/Others/Second/BERT Sentiment Analysis/Kaggle 1.6M/Final 1"
  model.load_state_dict(torch.load(os.path.join(direc_path, 'BERT_ft_epoch10.model'), map_location=torch.device('cuda')))
  return model 


def evaluate(dataloader_val, model):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []
  for batch in dataloader_val:
    batch = tuple(b.to('cuda') for b in batch)

    inputs = {
        'input_ids':batch[0],
        'attention_mask':batch[1],
        'labels':batch[2]
    }
    with torch.no_grad(): #no gradient change here, so no training occurs
      outputs = model(**inputs)

    loss = outputs[0]
    logits = outputs[1]
    loss_val_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)
  
  loss_val_avg = loss_val_total/len(dataloader_val) 

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals


def test_with_input(text, label, model):
  text = np.array([text])
  label = np.array([label])
  encoded_data_test = tokenizer.batch_encode_plus(
    text,
    add_special_tokens=True,
    return_attention_mask=True,
    max_length=256,
    padding=True,
    return_tensors='pt'
  )

  input_ids_test = encoded_data_test['input_ids']
  attention_masks_test = encoded_data_test['attention_mask']
  labels_test = torch.tensor(label)

  dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

  dataloader_test = DataLoader(
    dataset_test
  )
  _, predictions_test, true_vals_test = evaluate(dataloader_test, model)
  return predictions_test


def main():
  st.title('Streamlit Sentiment Analysis Web App')
  menu = ['Home']
  st.write(""" 
  ## ML Web App using BERT Classifier to detect happy/sad tweets


  """)
  st.subheader('Enter Sentence')
  user_input = st.text_input('', max_chars=256, value='')

  if st.button('Find'):
    user_input = preprocessing(user_input)
    model = load_model()
    label = 1
    pred = test_with_input(user_input, label, model)

    pred = np.argmax(pred)
    emo = ''
    if pred == 0:
      emo = 'sad'
    elif pred == 1:
      emo = 'happy'
    st.write('Predicted Emotion: {}'.format(emo))



if __name__ == '__main__':
  main()

Overwriting app.py


In [ ]:
!ls

app.py	gdrive	sample_data


### Authenticate

In [ ]:
!ngrok authtoken xxxxxxxxxxx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


### Run file with streamlit

In [ ]:
!streamlit run app.py &>/dev/null&

### Create public url with ngrok

In [ ]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

'http://e23a7c0bd778.ngrok.io'

## Kill Ngrok

In [ ]:
ngrok.kill()